# Домашнее задание к лекции Работа с файловой системой и модули

1. Переведите содержимое файла purchase_log.txt в словарь purchases вида:
```
{‘1840e0b9d4’: ‘Продукты’, …}
```

2. Для каждого `user_id` в файле `visit_log.csv` определите третий столбец с категорией покупки (если покупка была, сам файл `visit_log.csv` изменять не надо). Запишите в файл `funnel.csv` визиты из файла `visit_log.csv`, в которых были покупки с указанием категории.

Учтите условия на данные:

- содержимое `purchase_log.txt` помещается в оперативную память компьютера
- содержимое `visit_log.csv - нет; используйте только построчную обработку этого файла

In [1]:
import json
from typing import Dict

# {user_id: category}
purchase_dict: Dict[str, str] = {}

In [4]:
def read_purchase_line(line, line_n=None):
    try:
        purchase = json.loads(line)
        user_id = purchase['user_id']
        category = purchase['category']
    except KeyError:
        print(f'Absent keys in {purchase}. Line {line_n}.')
        return None, None
    except JSONDecodeError:
        print(f'Broken Line: {line[80:]}. Line {line_n}.')
        return None, None
    return user_id, category
    
with open('./purchase_log.txt', 'r') as purchase_log_f:
    purchase_log_f.readline()
    for i, line in enumerate(purchase_log_f):
        user_id, category = read_purchase_line(line, i + 1)
        if user_id:
            purchase_dict[user_id] = category
    print(f'Read {i+1} data lines, written {len(purchase_dict)} in the purchase dict.')


Read 99999 data lines, written 99517 in the purchase dict.


In [6]:
def extend_visit_line(line, line_n):
    try:
        user_id, source = line.strip().split(',')
    except ValueError:
        print(f'Broken Line {line_n}: {line[:80]}')
        return None
    global purchase_dict # Not going to change it, just for readability
    if user_id in purchase_dict:
        category = purchase_dict[user_id]
        return ','.join((user_id, source, category))
    else:
        return None
    
    

with open('./visit_log.csv', 'r') as visit_f:
    writen = 0
    wrongs = 0
    with open('./funnel.csv', 'w') as funnel_f:
        header = visit_f.readline().strip()
        funnel_f.write(f'{header},category\n')
        read_counter = 0
        write_counter = 0
        wrong_counter = 0
        for i, line in enumerate(visit_f):
            ext_line = extend_visit_line(line, i + 1)
            if ext_line:
                funnel_f.write(ext_line)
                funnel_f.write('\n')
                writen += 1
            else:
                wrongs += 1
        print(f'Read: {i+1} lines, written: {writen} lines, wrong or absent: {wrongs} lines.')
    

Read: 199999 lines, written: 13798 lines, wrong or absent: 186201 lines.
